In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import cv2
import matplotlib.pyplot as plt
from sys import getsizeof
import gc
%matplotlib inline

In [2]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Conv2D, Activation, MaxPooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping, ReduceLROnPlateau
from keras.applications.resnet50 import preprocess_input

Using TensorFlow backend.


In [3]:
base_model = applications.ResNet50(include_top=False, weights='imagenet', input_tensor=None, input_shape=(200,200,3), pooling=None)


In [4]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 100, 100, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 100, 100, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 100, 100, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [5]:
for layer in base_model.layers[:-75]:
     layer.trainable = False

In [6]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512,activation='relu')(x)
x = Dense(8, activation='softmax')(x)
model = Model(base_model.input, x)

In [7]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 100, 100, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 100, 100, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 100, 100, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [8]:
from keras import metrics
def top_2_accuracy(y_true, y_pred):
    return metrics.top_k_categorical_accuracy(y_true, y_pred, k=2)
    

In [9]:
adam = optimizers.Adam(lr=0.0001,beta_1=0.9, beta_2=0.999, epsilon=k.epsilon(), decay=0.0)

In [10]:
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy',top_2_accuracy])

In [11]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor = 0.5,patience = 1, min_lr = 0.00001, verbose = 1)

In [12]:
train_directory = "Images/train_images_cropped_downsampled"
t_size = (200, 200)
b_size = 16
test_directory = "Images/validation_images_cropped_downsampled"

In [24]:
train_gen = ImageDataGenerator(
preprocessing_function=preprocess_input,
horizontal_flip = True,
rotation_range = 30,
zoom_range = 0.2)

test_gen = ImageDataGenerator(
preprocessing_function=preprocess_input,
horizontal_flip = True)

train_generator = train_gen.flow_from_directory(
train_directory,
target_size = t_size,
batch_size = b_size,
class_mode = "categorical")

validation_generator = test_gen.flow_from_directory(
test_directory,
target_size = t_size,
batch_size = b_size,
class_mode = "categorical")



Found 61136 images belonging to 8 classes.
Found 800 images belonging to 8 classes.


In [14]:
checkpoint = ModelCheckpoint(filepath='saved_models/model.weights.best.down_sampled_3101.{epoch:02d}-{val_loss:.2f}.hdf5', verbose=1, save_best_only=True)

In [15]:
history7 = model.fit_generator(train_generator, epochs=10, validation_data=validation_generator, verbose=2, 
                               callbacks=[checkpoint,reduce_lr], steps_per_epoch=3821, validation_steps=50)

Epoch 1/10
Epoch 00001: val_loss improved from inf to 1.46089, saving model to saved_models/model.weights.best.down_sampled_3101.01-1.46.hdf5
 - 1403s - loss: 1.3412 - acc: 0.5058 - top_2_accuracy: 0.7161 - val_loss: 1.4609 - val_acc: 0.4612 - val_top_2_accuracy: 0.6975
Epoch 2/10
Epoch 00002: val_loss improved from 1.46089 to 1.42816, saving model to saved_models/model.weights.best.down_sampled_3101.02-1.43.hdf5
 - 1381s - loss: 1.1635 - acc: 0.5697 - top_2_accuracy: 0.7759 - val_loss: 1.4282 - val_acc: 0.4662 - val_top_2_accuracy: 0.7087
Epoch 3/10
Epoch 00003: val_loss improved from 1.42816 to 1.39946, saving model to saved_models/model.weights.best.down_sampled_3101.03-1.40.hdf5
 - 1382s - loss: 1.0971 - acc: 0.5936 - top_2_accuracy: 0.7966 - val_loss: 1.3995 - val_acc: 0.4863 - val_top_2_accuracy: 0.7087
Epoch 4/10
Epoch 00004: val_loss did not improve
 - 1380s - loss: 1.0572 - acc: 0.6102 - top_2_accuracy: 0.8079 - val_loss: 1.4317 - val_acc: 0.4763 - val_top_2_accuracy: 0.7300
E

In [14]:
checkpoint = ModelCheckpoint(filepath='saved_models/model.weights.best.down_sampled_0102.{epoch:02d}-{val_acc:.2f}.hdf5', verbose=1, save_best_only=True)

In [15]:
history8 = model.fit_generator(train_generator, epochs=15, validation_data=validation_generator, verbose=2, 
                               callbacks=[checkpoint,reduce_lr], steps_per_epoch=3821, validation_steps=50)

Epoch 1/15
Epoch 00001: val_loss improved from inf to 1.54794, saving model to saved_models/model.weights.best.down_sampled_0102.01-0.45.hdf5
 - 1400s - loss: 1.3408 - acc: 0.5035 - top_2_accuracy: 0.7158 - val_loss: 1.5479 - val_acc: 0.4500 - val_top_2_accuracy: 0.6800
Epoch 2/15
Epoch 00002: val_loss improved from 1.54794 to 1.47615, saving model to saved_models/model.weights.best.down_sampled_0102.02-0.45.hdf5
 - 1381s - loss: 1.1628 - acc: 0.5708 - top_2_accuracy: 0.7759 - val_loss: 1.4762 - val_acc: 0.4537 - val_top_2_accuracy: 0.6875
Epoch 3/15
Epoch 00003: val_loss improved from 1.47615 to 1.37216, saving model to saved_models/model.weights.best.down_sampled_0102.03-0.51.hdf5
 - 1381s - loss: 1.0986 - acc: 0.5946 - top_2_accuracy: 0.7980 - val_loss: 1.3722 - val_acc: 0.5050 - val_top_2_accuracy: 0.7312
Epoch 4/15
Epoch 00004: val_loss did not improve
 - 1384s - loss: 1.0538 - acc: 0.6104 - top_2_accuracy: 0.8119 - val_loss: 1.4014 - val_acc: 0.5125 - val_top_2_accuracy: 0.7025
E

In [13]:
#now testing on full validation data making use of different performance measure
#first loading the weights from previously saved models
model.load_weights(filepath="saved_models/model.weights.best.down_sampled_3101.08-1.34.hdf5")

In [14]:
full_validation_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
full_validation_generator = full_validation_gen.flow_from_directory(
"Images/full_validation",
target_size = t_size,
batch_size = 1,
class_mode=None,
shuffle=False)

Found 3999 images belonging to 1 classes.


In [15]:
#now do the prediction
predictions = model.predict_generator(full_validation_generator,steps=3999)

In [29]:
top_1_pred = []
for i in predictions:
    top_1_pred.append(np.argmax(i))
top_1_pred = np.array(top_1_pred)

In [30]:
top_2_pred = []
for i in predictions:
    top_2_pred.append(np.argpartition(i,-2)[-2:])
top_2_pred = np.array(top_2_pred)

In [31]:
top_2_pred

array([[7, 1],
       [5, 6],
       [5, 6],
       ..., 
       [4, 3],
       [4, 6],
       [4, 3]], dtype=int64)

In [17]:
full_validation_set = pd.read_csv("datasets/full_validation_set.csv")

In [18]:
full_validation_set = full_validation_set.drop('Unnamed: 0',axis=1)

In [19]:
full_validation_set

,subDirectory_filePath,face_x,face_y,face_width,face_height,expression,valence,arousal
0,328/7dc825b5a55f0c7a877773868b883c550f513efdcb...,24,24,162,162,7,-0.319396,0.333914
1,427/0bc723f0a6a0cb33a4ae1ca53f203ff14d790e6e8b...,119,119,795,795,4,-0.103175,0.928571
2,874/24c566d363e0f05a6df344804a705957a71e2d1870...,46,46,313,313,7,-0.647914,0.653182
3,357/0af4eb252fe5dcf012229787b52fa2bdfe65e7e9fa...,88,88,590,590,2,-0.888889,-0.365079
4,481/ffba2d1395dcfe3f2a81850cbc1525f34969ec0789...,77,77,518,518,2,-0.928571,-0.095238
5,854/14558b183e1db89777afef50f22e2cb461b6567f41...,82,82,549,549,2,-0.772066,-0.204224
6,462/bb8fd77a0cd112dc280bcf90eb02d13e745d9a1d95...,91,91,612,612,7,-0.662989,0.648471
7,493/67ed933f3336366a3c9fa8dd1b6d8cfc2fdf446632...,13,13,296,296,2,-0.888889,-0.206349
8,849/1b95b83aadd670c369da4184ae0dac4aa7994c16e7...,29,29,639,639,1,0.563492,-0.063492
9,518/a990be317b7b4741996554770ff1045b64574c99c3...,31,31,210,210,4,-0.056073,0.901484


In [20]:
import os
image_names = os.listdir('Images/full_validation/unknown_class')

In [21]:
for i in range(len(image_names)):
    image_names[i] = image_names[i].replace('_','/')

In [24]:
true_labels = []
for name in image_names:
    temp = full_validation_set[full_validation_set['subDirectory_filePath'] == name]
    true_labels.append(temp.iloc[0]['expression'])


In [25]:
true_labels = np.array(true_labels)

In [68]:
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score, confusion_matrix

In [32]:
top_1_score = accuracy_score(true_labels,top_1_pred)

In [33]:
top_1_score

0.53913478369592394

In [34]:
def get_top2_score(true_labels, top_2_pred):
    total = true_labels.shape[0]
    success = 0
    for i in range(total):
        if true_labels[i] in top_2_pred[i]:
            success += 1
    return success/total

In [35]:
top_2_score = get_top2_score(true_labels, top_2_pred)

In [36]:
top_2_score

0.75743935983996

In [63]:
f1_score_top_1 = f1_score(y_true=true_labels, y_pred=top_1_pred, average=None)

In [64]:
f1_score_top_1

array([ 0.49027895,  0.71490846,  0.57632933,  0.56570931,  0.60491071,
        0.43490701,  0.51762683,  0.23865546])

In [66]:
kappa_top_1 = cohen_kappa_score(true_labels, top_1_pred)

In [67]:
kappa_top_1

0.47328560797518648

In [80]:
confusion_mat = confusion_matrix(true_labels, top_1_pred, labels=[0,1,2,3,4,5,6,7])

In [81]:
confusion_mat

array([[290,  23,  61,  55,  11,   2,  52,   6],
       [ 23, 410,  10,  34,   2,   4,   6,  11],
       [ 58,   4, 336,  32,  12,   8,  48,   2],
       [ 39,  25,  31, 325,  54,   3,  22,   1],
       [ 22,   7,  57, 105, 271,   9,  29,   0],
       [ 44,  15,  71,  37,  24, 152, 154,   2],
       [ 62,   8,  62,  31,  19,  15, 301,   2],
       [145, 155,  38,  30,   3,   7,  51,  71]])